In [1]:
import tensorflow as tf
import datetime

import numpy as np
import matplotlib.pyplot as plt

2023-05-10 11:40:23.726420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train, validation = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/train',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224),
    seed=815,
    validation_split=0.1,
    subset='both'
)

test = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/test',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224)
)

def prepare(dataset):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomTranslation(0.2, 0.2),
        tf.keras.layers.RandomZoom(0.2),
    ])

    augmented = dataset
    for _ in range(4):
        augmented = augmented.concatenate(
            dataset.map(
                lambda x, y: (data_augmentation(x, training=True), y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
        )
        
    return augmented.prefetch(buffer_size=tf.data.AUTOTUNE)

train = prepare(train)

Found 22555 files belonging to 25 classes.
Using 20300 files for training.
Using 2255 files for validation.
Found 2500 files belonging to 25 classes.


2023-05-10 11:40:25.488964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 11:40:25.492531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 11:40:25.492685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
def generate_model():
    base = tf.keras.applications.MobileNetV3Small(
        include_top=False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None
    )
    
    for layer in base.layers:
        layer.trainable = False
    
    model = tf.keras.models.Sequential()
    model.add(base)

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(25, activation='softmax'))

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001, 
        decay_steps=1000, 
        decay_rate=0.9
    )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    return model

model = generate_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
 dense (Dense)               (None, 128)               3612800   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 25)                3225      
                                                                 
Total params: 4,571,657
Trainable params: 3,632,537
Non-trainable params: 939,120
______________________________________

In [ ]:
name = 'training-test'
now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=f'logs/fit/{name}-{now}',
    write_steps_per_second=True
)
    
model.fit(
    train,
    validation_data=validation,
    epochs=10, 
    callbacks=[tensorboard_callback]
)

Epoch 1/10


2023-05-10 11:40:27.832272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_93' with dtype resource
	 [[{{node Placeholder/_93}}]]
2023-05-10 11:40:27.833096: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string and shape [20300]
	 [[{{node Placeholder/_19}}]]
/usr/local/lib/python3.8/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-05-10 11:40:30.204336: I tenso

3173/3175 [============================>.] - ETA: 0s - loss: 0.9978 - accuracy: 0.6912

2023-05-10 11:42:18.163869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2255]
	 [[{{node Placeholder/_0}}]]
2023-05-10 11:42:18.164050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2255]
	 [[{{node Placeholder/_4}}]]


3175/3175 [==============================] - 112s 34ms/step - loss: 0.9977 - accuracy: 0.6912 - val_loss: 0.6994 - val_accuracy: 0.7672
Epoch 2/10
3175/3175 [==============================] - 108s 34ms/step - loss: 0.6359 - accuracy: 0.7936 - val_loss: 0.5975 - val_accuracy: 0.8102
Epoch 3/10
1233/3175 [==========>...................] - ETA: 52s - loss: 0.4261 - accuracy: 0.8583

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=f'logs/evaluate/{name}-{now}',
    write_steps_per_second=True
)

model.evaluate(
    test,
    callbacks=[tensorboard_callback]
)